In [1]:
!pip install datasets jiwer accelerate -U

Logging to Hugging face Hub for Common_voice latest dataset for Fine Tuning
You can access your token by going to Hugging face website > profile(create if not created) > access Tokens.

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

Loading the dataset from datasets library with load_dataset Function

In [ ]:
from datasets import load_dataset

common_voice_train= load_dataset("mozilla-foundation/common_voice_17_0", "hi", split="train",trust_remote_code=True)
common_voice_test= load_dataset("mozilla-foundation/common_voice_17_0", "hi", split="test",trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]


Reading metadata...: 4689it [00:00, 82450.88it/s]


Generating validation split: 0 examples [00:00, ? examples/s]


Reading metadata...: 2428it [00:00, 96198.54it/s]


...


Reading metadata...: 4044it [00:00, 104693.21it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 775it [00:00, 83788.78it/s]


Generating validated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 10329it [00:00, 95209.03it/s] 


Checking the data for both train and test

In [ ]:
common_voice_train.column_names

['client_id',
 'path',
 'audio',
 'sentence',
 'up_votes',
 'down_votes',
 'age',
 'gender',
 'accent',
 'locale',
 'segment',
 'variant']

In [ ]:
common_voice_test.column_names

['client_id',
 'path',
 'audio',
 'sentence',
 'up_votes',
 'down_votes',
 'age',
 'gender',
 'accent',
 'locale',
 'segment',
 'variant']

Removing Useless columns.

In [ ]:
common_voice_train=common_voice_train.remove_columns(['accent','age','client_id','down_votes','gender','locale','segment','up_votes','variant'])
common_voice_test=common_voice_test.remove_columns(['accent','age','client_id','down_votes','gender','locale','segment','up_votes','variant'])

Again Checking Column

In [ ]:
common_voice_test.column_names

['path', 'audio', 'sentence']

Checking the data with path array[audio] and sentence

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[0])
    display(HTML(df.to_html()))

show_random_elements(common_voice_train.remove_columns(['path']), num_examples=5)

,audio,sentence
array,"[5.816113682013476e-26, -1.4863401631812216e-25, -9.370405376577267e-26, 0.0, -4.52364397489937e-26, 1.2924697071141057e-26, -1.550963648536927e-25, -5.00832011506716e-26, -1.2924697071141057e-26, 2.908056841006738e-26, -5.169878828456423e-26, 1.2924697071141057e-26, -5.816113682013476e-26, -2.908056841006738e-26, -6.462348535570529e-27, 2.1810426307550534e-26, -1.7266587493477506e-26, 5.2506581851510546e-27, 8.077935669463161e-27, 7.270142102516845e-27, 1.857925203976527e-26, 3.937993638863291e-26, 1.2298657056757662e-25, 1.2076513825847426e-25, 2.2012374699287113e-26, 8.077935669463161e-26, 6.825855640696371e-26, -1.3767831606641275e-25, -7.270142102516845e-27, 4.8467614016778965e-26, 1.40556080648659e-25, 6.947024675738318e-26, 4.765982044983265e-26, 3.877409121342317e-26, -1.2924697071141057e-26, -1.809457589959748e-25, -7.108583389127582e-26, -5.169878828456423e-26, -6.462348535570529e-26, -5.816113682013476e-26, 1.2924697071141057e-26, -1.6802106192483375e-25, -2.5849394142282115e-25, -3.618915179919496e-25, 0.0, 5.169878828456423e-26, -7.754818242684634e-26, 1.3732490638087374e-26, 0.0, -1.6155871338926322e-27, -9.693522803355793e-26, -1.8256134612986744e-25, -7.229752424169529e-26, 1.0501316370302109e-26, 2.7464981276174747e-26, -9.34011311781678e-26, -8.077935669463161e-27, -4.119747191426212e-26, -6.987414354085634e-26, 1.1228330580553794e-25, -1.2641969322709847e-25, -4.806371723330581e-26, -3.917798799689633e-26, -2.645523931749185e-26, -5.0487097934144756e-26, -1.0097419586828951e-25, -8.603001487978266e-26, -8.077935669463161e-28, -8.643391166325582e-26, -2.9623304712859435e-26, 1.3732490638087374e-26, 2.5849394142282115e-26, -1.454028420503369e-26, -7.350921459211476e-26, -3.75624008630037e-26, -1.2238072539236689e-25, -2.9403685836845906e-25, -3.2311742677852644e-25, -4.8467614016778965e-26, -2.908056841006738e-26, -1.141513284291013e-25, -8.724170523020214e-26, 1.6155871338926322e-26, 2.463770379186264e-26, 3.069615554396001e-26, -3.715850407953054e-26, 5.169878828456423e-26, 3.2311742677852644e-26, 8.804949879714845e-26, 8.885729236409477e-26, 9.693522803355793e-26, 3.0292258760486853e-27, 1.001664023013432e-25, 3.2311742677852644e-26, -2.5849394142282115e-26, -5.169878828456423e-26, 2.261821987449685e-26, -1.809457589959748e-25, 0.0, 1.550963648536927e-25, ...]",हमने उसका जन्मदिन मनाया।
path,/root/.cache/huggingface/datasets/downloads/extracted/ceb0ef62c8077220b6489fcb013e4324c9ffab8e0fe29fd888022eea20269a56/hi_train_0/common_voice_hi_26008353.mp3,हमने उसका जन्मदिन मनाया।
sampling_rate,48000,हमने उसका जन्मदिन मनाया।


Characters to Ignore

In [ ]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower() + ' '
    return batch

In [ ]:
common_voice_train  = common_voice_train.map(remove_special_characters)
common_voice_test  = common_voice_test.map(remove_special_characters)

Map:   0%|          | 0/4689 [00:00<?, ? examples/s]

Map:   0%|          | 0/3154 [00:00<?, ? examples/s]

In [ ]:
def extract_all_chars(batch):
    all_text = " ".join(batch["sentence"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

Map:   0%|          | 0/4689 [00:00<?, ? examples/s]

Map:   0%|          | 0/3154 [00:00<?, ? examples/s]

In [ ]:
vocab_list=list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [ ]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'ख़': 0,
 'o': 1,
 'ऋ': 2,
 'u': 3,
 'ऊ': 4,
 '’': 5,
 'क': 6,
 'े': 7,
 'श': 8,
 'z': 9,
 'i': 10,
 'औ': 11,
 'ढ': 12,
 'p': 13,
 'य': 14,
 'फ़': 15,
 'ी': 16,
 'ठ': 17,
 "'": 18,
 'ख': 19,
 'j': 20,
 's': 21,
 'ल': 22,
 'ढ़': 23,
 'ई': 24,
 'ि': 25,
 'ो': 26,
 'n': 27,
 'ै': 28,
 'b': 29,
 'x': 30,
 'आ': 31,
 'ड़': 32,
 'q': 33,
 'व': 34,
 'ष': 35,
 'ञ': 36,
 'घ': 37,
 'द': 38,
 'ौ': 39,
 't': 40,
 'स': 41,
 ' ': 42,
 'फ': 43,
 '़': 44,
 'ु': 45,
 'भ': 46,
 'क़': 47,
 'e': 48,
 'ह': 49,
 'w': 50,
 'उ': 51,
 'न': 52,
 'd': 53,
 'ओ': 54,
 'ज़': 55,
 'ज': 56,
 '&': 57,
 'ॅ': 58,
 'l': 59,
 'ॉ': 60,
 'ध': 61,
 'ए': 62,
 'ः': 63,
 'ँ': 64,
 '–': 65,
 'थ': 66,
 'छ': 67,
 'ू': 68,
 'y': 69,
 'म': 70,
 'h': 71,
 'm': 72,
 '|': 73,
 'ण': 74,
 'ृ': 75,
 'इ': 76,
 'ट': 77,
 'a': 78,
 'र': 79,
 'अ': 80,
 '्': 81,
 '।': 82,
 'g': 83,
 'ड': 84,
 'त': 85,
 'v': 86,
 'r': 87,
 'ा': 88,
 'c': 89,
 'ं': 90,
 'ऑ': 91,
 'झ': 92,
 'प': 93,
 'k': 94,
 'ग': 95,
 'ब': 96,
 'ऐ': 97,
 'च': 98,
 'f': 99}

In [ ]:
vocab_dict['[UNK]']= len(vocab_dict)
vocab_dict['[PAD]']= len(vocab_dict)
len(vocab_dict)

102

Create Json file where vocabs are saved with their corresponding labels

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

Creating Tokenizer From Transformers Wav2Vec2Tokenizer

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

Create XLSR-Wav2-Vec2 Feature extractor

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

Creating Processor from transformers

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

#Preprocess the data

Checking first sample of the data

In [ ]:
common_voice_train[0]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/ceb0ef62c8077220b6489fcb013e4324c9ffab8e0fe29fd888022eea20269a56/hi_train_0/common_voice_hi_26008353.mp3',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/ceb0ef62c8077220b6489fcb013e4324c9ffab8e0fe29fd888022eea20269a56/hi_train_0/common_voice_hi_26008353.mp3',
  'array': array([ 5.81611368e-26, -1.48634016e-25, -9.37040538e-26, ...,
          1.06425901e-07,  4.46416450e-08,  2.61450239e-09]),
  'sampling_rate': 48000},
 'sentence': 'हमने उसका जन्मदिन मनाया। '}

Converting speech file to array for model

In [ ]:
import torchaudio

def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    batch['speech'] = speech_array[0].numpy()
    batch['sampling_rate'] = sampling_rate
    batch['target_text'] = batch['sentence']
    return batch

Maping with the Function

In [ ]:
common_voice_train = common_voice_train.map(speech_file_to_array_fn,remove_columns=common_voice_train.column_names)
common_voice_test = common_voice_test.map(speech_file_to_array_fn,remove_columns=common_voice_test.column_names)

Map:   0%|          | 0/4689 [00:00<?, ? examples/s]

Map:   0%|          | 0/3154 [00:00<?, ? examples/s]

In [ ]:
'''import librosa
import numpy as np

def resample(batch):
    batch["speech"] = librosa.resample(np.asarray(batch['speech']), orig_sr=48_000, target_sr=16_000)
    batch['sampling_rate'] = 16_000
    return batch'''

'''common_voice_train = common_voice_train.map(resample, num_proc=4)
common_voice_test = common_voice_test.map(resample,num_proc=4)'''

'common_voice_train = common_voice_train.map(resample, num_proc=4)\ncommon_voice_test = common_voice_test.map(resample,num_proc=4)'

Resampling the sampling rate from orginal sampling to target sampling rate =16_000 for model

In [ ]:
import IPython.display as ipd
import librosa
import numpy as np
def resample(batch):
    orig_sr = batch['sampling_rate']
    target_sr = 16000
    if orig_sr != target_sr:
        batch["speech"] = librosa.resample(np.asarray(batch['speech']), orig_sr=orig_sr, target_sr=target_sr)
        batch['sampling_rate'] = target_sr
    return batch

common_voice_train = common_voice_train.map(resample, num_proc=16)
common_voice_test = common_voice_test.map(resample, num_proc=16)



Map (num_proc=16):   0%|          | 0/4689 [00:00<?, ? examples/s]

Accessing the speech

In [ ]:
first_entry = common_voice_train[1450]

ipd.display(ipd.Audio(data=first_entry['speech'], autoplay=True, rate=first_entry['sampling_rate'])) # Access 'speech' directly for audio data

Accessing random speech from the training data

In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(1, len(common_voice_train)-1)
ipd.Audio(data=common_voice_train[rand_int]["speech"], autoplay=True, rate=16_000)


accessing random speech from the testing data

In [ ]:
rand_int = random.randint(1, len(common_voice_test)-1)
ipd.Audio(data=common_voice_test[rand_int]["speech"], autoplay=True, rate=16_000)

Accessing the Target Text, Input array shape, sampling rate of the data

In [ ]:
rand_int = random.randint(0, len(common_voice_test))

print("Target Text :", common_voice_test[rand_int]["target_text"])
print("Input array shape :",np.asarray(common_voice_test[rand_int]["speech"]).shape)
print("sampling Rate:", common_voice_test[rand_int]["sampling_rate"])


print("Target Text :", common_voice_train[rand_int]["target_text"])
print("Input array shape :",np.asarray(common_voice_train[rand_int]["speech"]).shape)
print("sampling Rate:", common_voice_train[rand_int]["sampling_rate"])

Preparing the dataset

In [ ]:
def prepare_dataset(batch):
  assert(
      len(set(batch["sampling_rate"])) == 1
  ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."
  batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values
  with processor.as_target_processor():
    batch["labels"] = processor(batch["target_text"]).input_ids
  return batch

Dataset mapping

In [ ]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names,batch_size=8, num_proc=8,batched=True)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names,batch_size=8, num_proc=8,batched=True)

#Training setting up Trainer

This code dynamically pads the input and labels to match their lengths, and then moves them to the GPU for training.







In [ ]:
import torch
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass

class DataCollactorCTCWithPadding:
     """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
     """
     processor:Wav2Vec2Processor
     padding: Union[bool, str] = True
     max_length: Optional[int] = None
     max_length_labels: Optional[int] = None
     pad_to_multiple_of: Optional[int] = None
     pad_to_multiple_of_labels: Optional[int] = None

     def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
         # split inputs and labels since they have to be of different lenghts and need
         # different padding methods
         input_features = [{"input_values": feature["input_values"]} for feature in features]
         label_features = [{"input_ids": feature["labels"]} for feature in features]

         batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

         with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )
       # replace padding with -100 to ignore loss correctly
         labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

         batch["labels"] = labels

         return batch

preparing data with CTC (Connectionist Temporal Classification) padding enabled, likely for sequence processing tasks like speech or text recognition.

In [ ]:
data_collator = DataCollactorCTCWithPadding(processor=processor, padding=True)

Loading the WER Metric for Evaluation

In [ ]:
from datasets import load_metric
wer_metric = load_metric("wer")

Computing Metrics for Evaluation Using Predictions and Labels

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

**Loading Wav2Vec2ForCTC Model and Configuring for CUDA Execution**

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53",
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    gradient_checkpointing=True,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
).to("cuda")

freezing (disabling training updates for) the feature extractor part of the model

In [ ]:
model.freeze_feature_extractor()

**Model Training Configuration Essentials**

In [ ]:

from transformers import TrainingArguments

training_args = TrainingArguments(
  #output_dir="/content/gdrive/MyDrive/wav2vec2-large-xlsr-hindi",
  output_dir="./wav2vec2-large-xlsr-hindi",
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  eval_strategy="steps",
  num_train_epochs=30,
  fp16=True,
  save_steps=400, #this would mean every 400 steps model gets saved which also means Google drive gets full
  eval_steps=400,
  logging_steps=400,
  #learning_rate=3e-4,
  learning_rate=0.1, # this is just for demo
  warmup_steps=500,
  save_total_limit=2,
)

**Initializing Trainer for Model Training and Evaluation**

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

starts the training process for the model using the specified configurations and datasets.

In [ ]:
trainer.train()

In [ ]:
# Save the model and tokenizer
model.save_pretrained("./wav2vec2-large-xlsr-hindi")
processor.save_pretrained("./wav2vec2-large-xlsr-hindi")


Testing Part

In [ ]:
# prompt: inputs = processor(test_dataset["speech"][:2], sampling_rate=16_000, return_tensors="pt", padding=True) want this type of code for common_voice_test in my fine tuning process

inputs = processor(common_voice_test["speech"][:2], sampling_rate=16_000, return_tensors="pt", padding=True)


In [ ]:
with torch.no_grad():
  logits = model(inputs.input_values, attention_mask=inputs.attention_mask).logits

In [ ]:
predicted_ids = torch.argmax(logits, dim=-1)

In [ ]:
print("Prediction:", processor.batch_decode(predicted_ids))
print("Reference:", test_dataset["sentence"][:2])